<a href="https://colab.research.google.com/github/LeVuPhuoc/AI-Robotics/blob/main/Classification_15_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import glob

import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/AI/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

# # !unzip valid_flowers.zip
# # !unzip train_flowers.zip
# # !rm -r black* calendula
# !ls


Classification_15_flowers.ipynb  train_flowers.zip  valid_flowers.zip
train_flowers			 valid_flowers


In [ ]:
def plot_history(history_fine):
  f1 = history_fine.history['accuracy']
  val_f1 = history_fine.history['val_accuracy']

  loss = history_fine.history['loss']
  val_loss = history_fine.history['val_loss']

  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)
  plt.plot(f1, label='accuracy')
  plt.plot(val_f1, label='Validation accuracy')
  plt.legend(loc='lower right')
  plt.title('Accuracy')

  plt.subplot(2, 1, 2)
  plt.plot(loss, label='Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Loss')
  plt.xlabel('epoch')
  plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224, 3)
N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 8


train_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2,
)
test_datagen = ImageDataGenerator(
    rescale=1/255,
)

training_set = train_datagen.flow_from_directory(
    directory='train_flowers/',
    target_size=IMAGE_SHAPE[:2], 
    batch_size=32, 
    class_mode='categorical',
    subset='training',
)
validation_set = train_datagen.flow_from_directory(
    directory='train_flowers/',
    target_size=IMAGE_SHAPE[:2], 
    batch_size=32, 
    class_mode='categorical',
    subset='validation',
)

test_set = test_datagen.flow_from_directory(
    directory='valid_flowers/',
    target_size=IMAGE_SHAPE[:2], 
    batch_size=32, 
    class_mode='categorical'
)

NUM_CLASSES = len(training_set.class_indices)

In [ ]:
from tensorflow.keras.models import Model
from keras.layers import Conv2D, BatchNormalization, ReLU, Activation
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Dropout
from keras.layers import Flatten, Input
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam


model = Sequential()
model.add(Input(shape=(IMAGE_SHAPE)))
model.add(BatchNormalization())


model.add(Conv2D(filters=30,kernel_size=7,strides=1,padding='same',
                 kernel_initializer='he_uniform',))
model.add(Activation(activation='relu'))  
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.33))

model.add(Conv2D(filters=140,kernel_size=3,strides=1,padding='same',
                 kernel_initializer='he_uniform',))
model.add(Activation(activation='linear'))  
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate= 0.35))

model.add(Conv2D(filters=30,kernel_size=3,strides=2,padding='valid',
                 kernel_initializer='he_uniform',))
model.add(Activation(activation='relu'))  
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(rate= 0.42))
model.add(Flatten())

model.add(Dense(units=17,kernel_initializer='he_uniform',))
model.add(Activation(activation='relu'))  

model.add(Dropout(rate=0.27))
model.add(Dense(15,activation="softmax"))

# We compile our model with a sampled learning rate.
learning_rate = 0.001
beta_1 = 0.92
beta_2 = 0.92
epsilon =  1.178e-08


optimizer = Adam(learning_rate=learning_rate,
    beta_1=beta_1,
    beta_2=beta_2,
    epsilon=epsilon,
    amsgrad=False,
)
model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"],
)



history = model.fit(
    training_set,
    validation_data = validation_set,
    shuffle=True,
    batch_size=10,
    epochs=EPOCHS,
    verbose=True,

)

score = model.evaluate(test_set, verbose=0)
print("Loss={}, Accuracy={} ".format(score[0], score[1]))
plot_history(history)
plt.show()

